In [ ]:
#run these first:

from functools import lru_cache
import pandas as pd
import numpy as np
import scipy
from scipy.cluster.hierarchy import fclusterdata
import itertools
from scipy import stats
from typing_extensions import Counter


UL=["A",'B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
LL=list(map(chr, range(ord('a'), ord('z')+1)))
DD=["0",'1','2','3','4','5','6','7','8','9']
S=list(map(chr, range(33,48)))+list(map(chr, range(58,65)))+list(map(chr, range(91,97)))+list(map(chr, range(123,127)))
WS=[" "]

def abstractwords(word): #this function builds and returns a list that contains a list of each value's literal character and their class abstraction. [char,abstraction]
  wordlist=[]
  #word=word.strip('"') done for results purposes only for rayyan file (also done for rayyan results)
  for char in word:
    if char in UL:
      wordlist.append([char,"Ů"])
    elif char in LL:
      wordlist.append([char,"Ł"])
    elif char in DD:
      wordlist.append([char,"Đ"])
    elif char in S:
      wordlist.append([char,"Š"])
    elif char in WS:
      wordlist.append([char])
  return(wordlist)


coverage={}
g=[]

def getpatterns(d,word,limlength,maxlen): #this function returns generator of patterns
    #word=word.strip('"') #added for journal name. unreadable output so removing the " sign for both.
    if len(word)<limlength:
      for i in word:
        coverage.update({word:''})
      g.clear()
      combinations = itertools.product(*d)
      flat_list = [item for sublist in d for item in sublist]
      flat_list= (''.join(flat_list))
      for c in combinations:
          c=''.join(c)
          g.append(c)
      coverage.update({word:g[:]})
      return(g)
    elif len(word)>limlength and len(word)<maxlen:
      a=word.split() #splitting by words.
      longword=[]
      for i in a:
        j=i
        for char in i:
          if char in UL:
            j=i.replace(char,"Ů")
          elif char in LL:
            j=j.replace(char,"Ł")
          elif char in DD:
            j=j.replace(char,"Đ")
          elif char in S:
            j=j.replace(char,"Š")
          #elif char in WS:  chose not to abstract white spaces because it's just one value so creating redundant patterns.
            #j.replace(char,"⊔")
        longword.append([i,j])
      g.clear()
      combinations = itertools.product(*longword)
      for c in combinations:
        c=(' '.join(c))
        g.append(c)
      coverage.update({word:g[:]})
      return(g)

    else:
      verylongword=[]
      g.clear()
      j=word
      for char in word:
        if char in UL:
          j=j.replace(char,"Ů")
        elif char in LL:
          j=j.replace(char,"Ł")
        elif char in DD:
          j=j.replace(char,"Đ")
        elif char in S:
          j=j.replace(char,"Š")
      #print(word,j)
      g.append(word)
      g.append(j)
      coverage.update({word:g[:]})
      return(g)



b={}
def getdictionary(x,count_of_word): #this function checks if a pattern exists in the dictionary, and if it does not, creates a new entry. if it does exist, adds to counter.
  for i in x:
    counter=b.get(i,count_of_word) # in the beginning, b is an empty dict. counter checks if i, a pattern in list/dict x is in b, if it is, we assign the current freq to the counter, otherwise, we assign 1 to a new key.
    if i in b:
      counter=counter+count_of_word
    b.update({i:counter})
  return(b)

def getscore(b,genlevelcostmultiplier,minpatmultiplier,acc_threshold,lencomparison,pvalue,smallnfpvalue, smallnfnvalue):
  sorted_dict = dict(sorted(b.items(), key=lambda x:x[1],reverse=True))
  sorted_dict2=sorted_dict.items()
  #genlevelcostmultiplier=1.2
  for i in sorted_dict.items():
    genlevel=0
    for char in i[0]:
      if char in ('ŮŁĐŠ⊔'):
        genlevel=genlevel+1

    score=((i[1])*(1/((genlevel*genlevelcostmultiplier)+1)))
    sorted_dict.update({i[0]:score})
  sorted_dict = dict(sorted(sorted_dict.items(), key=lambda x:x[1],reverse=True))
  return(loopthrough(sorted_dict,minpatmultiplier,acc_threshold,lencomparison,pvalue,smallnfpvalue, smallnfnvalue))


def loopthrough(sorted_dict,minpatmultiplier,acc_threshold,lencomparison,pvalue,smallnfpvalue, smallnfnvalue):
  def get_rows(i,patternlist):
    coverage_values=list(coverage.values())
    subtractthis=[]
    saveforlater=[]
    for row in coverage_values:
      if patternlist[i] in row:
        subtractthis.append([x for x in row if x != patternlist[i]])
        saveforlater.append(row[0])
    pattern_values.update({patternlist[i]:saveforlater})
    return subtractthis

  def get_outliers(patternlist,minpatmultiplier,acc_threshold,lencomparison,pvalue,smallnfpvalue, smallnfnvalue):
    freq_cov={}
    outlier_patterns=[]
    dominant_patterns=[]
    valid_values=[]
    outlier_values=[]
    final_patterns={}
    final_values={}
    accumulate=0
    #acc_threshold=0.95
    for i in patternlist:
      freq_cov.update({i:b.get(i)})
    total=sum(freq_cov.values())
    freq_cov_2=list(freq_cov.values())
    mincov=(np.mean(freq_cov_2)/total)*minpatmultiplier
    for value in freq_cov.items():
      coveragevalue=(value[1]/total)
      if coveragevalue>mincov and accumulate<acc_threshold:
        dominant_patterns.append(value[0])
        accumulate=accumulate+coveragevalue
      else:
        outlier_patterns.append(value[0])
      print(coveragevalue,mincov,accumulate)
    for dom_pat in dominant_patterns:
      valid_values.append(pattern_values.get(dom_pat))
    for out_pat in outlier_patterns:
      outlier_values.append(pattern_values.get(out_pat))
    valid_values = [item for sublist in valid_values for item in sublist]
    outlier_values= [item for sublist in outlier_values for item in sublist]
    final_patterns.update({'dominant': dominant_patterns})
    final_patterns.update({'outlier patterns': outlier_patterns})
    final_values.update({'valid':valid_values})
    final_values.update({'outliers':outlier_values})
    return(get_distances(freq_cov,patternlist,outlier_patterns,dominant_patterns, final_values,final_patterns,lencomparison,pvalue,smallnfpvalue, smallnfnvalue))


  def get_distances(freq_cov,patternlist,outlier_patterns,dominant_patterns,final_values, final_patterns,lencomparison,pvalue,smallnfpvalue, smallnfnvalue): #calculates distances using class-weighted measure between pairwise dominant-dominant... and dominant-outlier. performs statistical z test to determine if significant results.
    print(freq_cov)
    domdom=itertools.product(dominant_patterns,dominant_patterns)
    domout=itertools.product(dominant_patterns,outlier_patterns)
    domoutscores={}
    domdomscores={}
    fpposition=[]
    fppositiondomdom=[]
    false_positives=[]
    false_negatives=[]
    for i in domout:
      if (len(i[0])<lencomparison) and (len(i[1])<lencomparison): #similarity measure length limitation
        try:
          domoutscores.update({i:AZWeightedLevenshtein.distance(i[0],i[1])})
        except:
          if len(i[0]) != len(i[0].encode()):
            print(i,"not ascii")
    for i in domdom:
      if (len(i[0])<lencomparison) and (len(i[1])<lencomparison): #limiting pairwise distance calculation for efficiency
        try:
          domdomscores.update({i:AZWeightedLevenshtein.distance(i[0],i[1])})
        except:
          if len(i[0]) != len(i[0].encode()):
            print(i,"not ascii")
    domoutscoreslist=list(domoutscores.values())
    domoutarr = np.array(domoutscoreslist)
    domoutz=stats.zscore(domoutarr)
    if len(domoutz)>15:
      for index,i in enumerate(domoutz):
        if i <0 and scipy.stats.norm.sf(abs(i)) <pvalue:
          fpposition.append(index)
      for i in fpposition:
        if list(domoutscores.keys())[i][1] not in false_positives:
          false_positives.append(list(domoutscores.keys())[i][1])
    else:
       for item in domoutscores.items():
         if (item[1])<smallnfpvalue and item[0][1] not in false_positives:
           false_positives.append(item[0][1])
    domdomscoreslist=list(domdomscores.values())
    domdomarr = np.array(domdomscoreslist)
    domdomz=stats.zscore(domdomarr)
    if len(domdomz)>15:
      for index,i in enumerate(domdomz):
       if i >0 and scipy.stats.norm.sf(abs(i)) <pvalue:
        fppositiondomdom.append(index)
      for i in fppositiondomdom:
        if (freq_cov[list(domdomscores.keys())[i][0]]>freq_cov[list(domdomscores.keys())[i][1]]):
          if list(domdomscores.keys())[i][1] not in false_negatives:
           false_negatives.append(list(domdomscores.keys())[i][1])
        else:
          if list(domdomscores.keys())[i][0] not in false_negatives:
            false_negatives.append(list(domdomscores.keys())[i][0])
    else:
      for item in domdomscores.items():   #this part is added in case the number of pairwise comparisons is smaller than 15, making statistical tests less valid.
        if(item[1]>smallnfnvalue): #largest cost of substitution of one inter-class character.
          if (freq_cov.get(item[0][0])>freq_cov.get(item[0][1])):
            if item[0][1] not in false_negatives:
              false_negatives.append(item[0][1])
          else:
            if item[0][0] not in false_negatives:
              false_negatives.append(item[0][0])
    fp_values=[]
    fn_values=[]
    for fp_pat in false_positives:
        fp_values.append(pattern_values.get(fp_pat))
    for fn_pat in false_negatives:
        fn_values.append(pattern_values.get(fn_pat))
    fp_values= [item for sublist in fp_values for item in sublist]
    fn_values= [item for sublist in fn_values for item in sublist]
    final_values.update({"FP":fp_values})
    final_values.update({"FN":fn_values})
    final_patterns.update({"domdomscores: ": domdomscores})
    final_patterns.update({"domoutscores: ": domoutscores})
    final_patterns.update({"FP_pat": false_positives})
    final_patterns.update({"FN_pat": false_negatives})
    df=pd.DataFrame.from_dict(final_values,orient='index').transpose()
    print('values:',final_values, '\npatterns',final_patterns)
    df.to_csv('results_col.csv', index=False)

  coverage_values=list(coverage.values())
  patternlist=list(sorted_dict.keys())
  plistset=set(patternlist)
  pattern_values={}
  for i in range (len(sorted_dict)):
    if i <len(patternlist):
      remainingrow=get_rows(i,patternlist)
      remainingrowflat = [item for sublist in remainingrow for item in sublist]
      for m in remainingrowflat:
        if m in plistset:
          patternlist.remove(m)
          plistset.remove(m)
    else:
      break
  return(get_outliers(patternlist,minpatmultiplier,acc_threshold,lencomparison,pvalue,smallnfpvalue, smallnfnvalue))

In [ ]:
pip install -U strsimpy #MIT License Copyright (c) 2018 luozhouyang; Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the Software), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 2.0 MB/s eta 0:00:00


In [ ]:

from functools import reduce
from strsimpy.string_distance import StringDistance


def default_insertion_cost(char):
    return 1.0
def default_deletion_cost(char):
    return 1.0
def default_substitution_cost(char_a, char_b):
    return 1.0

class AZWeightedLevenshtein(StringDistance):
    def __init__(self,
                 substitution_cost_fn=default_substitution_cost,
                 insertion_cost_fn=default_insertion_cost,
                 deletion_cost_fn=default_deletion_cost,
                 ):
        self.substitution_cost_fn = substitution_cost_fn
        self.insertion_cost_fn = insertion_cost_fn
        self.deletion_cost_fn = deletion_cost_fn

    def distance(self, s0, s1):
        if s0 is None:
            raise TypeError("Argument s0 is NoneType.")
        if s1 is None:
            raise TypeError("Argument s1 is NoneType.")
        if s0 == s1:
            return 0.0
        if len(s0) == 0:
            return reduce(lambda cost, char: cost + self.insertion_cost_fn(char,s0), s1, 0)
        if len(s1) == 0:
            return reduce(lambda cost, char: cost + self.deletion_cost_fn(char,s0), s0, 0)

        v0, v1 = [0.0] * (len(s1) + 1), [0.0] * (len(s1) + 1)
        v0[0] = 0
        for i in range(1, len(v0)):
            v0[i] = v0[i - 1] + self.insertion_cost_fn(s1[i - 1],s0)

        for i in range(len(s0)):
            s0i = s0[i]
            deletion_cost = self.deletion_cost_fn(s0i,s0)
            v1[0] = v0[0] + deletion_cost

            for j in range(len(s1)):
                s1j = s1[j]
                cost = 0
                if s0i != s1j:
                    cost = self.substitution_cost_fn(s0i, s1j)
                insertion_cost = self.insertion_cost_fn(s1j,s0)
                v1[j + 1] = min(v1[j] + insertion_cost, v0[j + 1] + deletion_cost, v0[j] + cost)
            v0, v1 = v1, v0

        return v0[len(s1)]





from strsimpy.weighted_levenshtein import WeightedLevenshtein


def insertion_cost(char,word):
    if (word[len(word)-1] in LL or word[len(word)-1]=='Ł')  and (char in LL or char=="Ł"):
      return 1.5
    elif (word[len(word)-1] in LL or word[len(word)-1]=='Ł') and (char not in LL or char !="Ł"):
      return 6
    if (word[len(word)-1] in UL or word[len(word)-1]=='Ů') and (char in UL or char=="Ů"):
      return 1.5
    elif (word[len(word)-1] in UL or word[len(word)-1]=='Ů') and (char not in UL or char !="Ů"):
      return 6
    if (word[len(word)-1] in DD or word[len(word)-1]=='Đ') and (char in DD or char=="Đ"):
      return 1.5
    elif (word[len(word)-1] in DD or word[len(word)-1]=='Đ') and (char not in DD or char !="Đ"):
      return 6
    if (word[len(word)-1] in S or word[len(word)-1]=='Š') and (char in S or char=="Š"):
      return 1.5
    elif (word[len(word)-1] in S or word[len(word)-1]=='Š') and (char not in S or char !="Š"):
      return 6
    if (word[len(word)-1] in WS or word[len(word)-1]=='⊔') and (char in WS or char=="⊔"):
      return 1.5
    elif (word[len(word)-1] in WS or word[len(word)-1]=='⊔') and (char not in WS or char !="⊔"):
      return 6

def deletion_cost(char,word):
    if (word[len(word)-2] in LL or word[len(word)-2]=='Ł')  and (char in LL or char=="Ł"):
      return 1.5
    elif (word[len(word)-2] in LL or word[len(word)-2]=='Ł') and (char not in LL or char !="Ł"):
      return 6
    if (word[len(word)-2] in UL or word[len(word)-2]=='Ů') and (char in UL or char=="Ů"):
      return 1.5
    elif (word[len(word)-2] in UL or word[len(word)-2]=='Ů') and (char not in UL or char !="Ů"):
      return 6
    if (word[len(word)-2] in DD or word[len(word)-2]=='Đ') and (char in DD or char=="Đ"):
      return 1.5
    elif (word[len(word)-2] in DD or word[len(word)-2]=='Đ') and (char not in DD or char !="Đ"):
      return 6
    if (word[len(word)-2] in S or word[len(word)-2]=='Š') and (char in S or char=="Š"):
      return 1.5
    elif (word[len(word)-2] in S or word[len(word)-2]=='Š') and (char not in S or char !="Š"):
      return 6
    if (word[len(word)-2] in WS or word[len(word)-2]=='⊔') and (char in WS or char=="⊔"):
      return 1.5
    elif (word[len(word)-2] in WS or word[len(word)-2]=='⊔') and (char not in WS or char !="⊔"):
      return 6


def substitution_cost(char_a, char_b):
    if (char_a == "Ł" and char_b in LL) or (char_a in LL and char_b =="Ł"):
        return 0.5
    elif (char_a == "Ł" and char_b in UL) or (char_a in UL and char_b =="Ł"):
        return 2.5
    elif (char_a == "Ł" and char_b =="Ů") or (char_a =="Ů" and char_b =="Ł"):
        return 2
    elif (char_a == "Ł" and char_b in DD) or (char_a in DD and char_b =="Ł"):
        return 7.5
    elif (char_a == "Ł" and char_b =="Đ") or (char_a =="Đ" and char_b =="Ł"):
        return 7
    elif (char_a == "Ł" and char_b in S) or (char_a in S and char_b =="Ł"):
        return 7.5
    elif (char_a == "Ł" and char_b =="Š") or (char_a =="Š" and char_b =="Ł"):
        return 7
    elif (char_a == "Ł" and char_b in WS) or (char_a in WS and char_b =="Ł"):
        return 7.5
    elif (char_a == "Ł" and char_b =="⊔") or (char_a =="⊔" and char_b =="Ł"):
        return 7

    if (char_a == "Ů" and char_b in UL) or (char_a in UL and char_b =="Ů"):
        return 0.5
    elif (char_a == "Ů" and char_b in LL) or (char_a in LL and char_b =="Ů"):
        return 2.5
    elif (char_a == "Ů" and char_b in DD) or (char_a in DD and char_b =="Ů"):
        return 7.5
    elif (char_a == "Ů" and char_b =="Đ") or (char_a =="Đ" and char_b =="Ů"):
        return 7
    elif (char_a == "Ů" and char_b in S) or (char_a in S and char_b =="Ů"):
        return 7.5
    elif (char_a == "Ů" and char_b =="Š") or (char_a =="Š" and char_b =="Ů"):
        return 7
    elif (char_a == "Ů" and char_b in WS) or (char_a in WS and char_b =="Ů"):
        return 7.5
    elif (char_a == "Ů" and char_b =="⊔") or (char_a =="⊔" and char_b =="Ů"):
        return 7

    if (char_a == "Đ" and char_b in DD) or (char_a in DD and char_b =="Đ"):
        return 0.5
    elif (char_a == "Đ" and char_b in LL) or (char_a in LL and char_b =="Đ"):
        return 7.5
    elif (char_a == "Đ" and char_b in UL) or (char_a in UL and char_b =="Đ"):
        return 7.5
    elif (char_a == "Đ" and char_b in S) or (char_a in S and char_b =="Đ"):
        return 6.5
    elif (char_a == "Đ" and char_b =="Š") or (char_a =="Š" and char_b =="Đ"):
        return 6
    elif (char_a == "Đ" and char_b in WS) or (char_a in WS and char_b =="Đ"):
        return 6.5
    elif (char_a == "Đ" and char_b =="⊔") or (char_a =="⊔" and char_b =="Đ"):
        return 6

    if (char_a == "Š" and char_b in S) or (char_a in S and char_b =="Š"):
        return 0.5
    elif (char_a == "Š" and char_b in LL) or (char_a in LL and char_b =="Š"):
        return 7.5
    elif (char_a == "Š" and char_b in UL) or (char_a in UL and char_b =="Š"):
        return 7.5
    elif (char_a == "Š" and char_b in DD) or (char_a in DD and char_b =="Š"):
        return 6.5
    elif (char_a == "Š" and char_b in WS) or (char_a in WS and char_b =="Š"):
        return 6.5
    elif (char_a == "Š" and char_b =="⊔") or (char_a =="⊔" and char_b =="Š"):
        return 6

    if (char_a == "⊔" and char_b in WS) or (char_a in WS and char_b =="⊔"):
        return 0.5
    elif (char_a == "⊔" and char_b in LL) or (char_a in LL and char_b =="⊔"):
        return 7.5
    elif (char_a == "⊔" and char_b in UL) or (char_a in UL and char_b =="⊔"):
        return 7.5
    elif (char_a == "⊔" and char_b in DD) or (char_a in DD and char_b =="⊔"):
        return 6.5
    elif (char_a == "⊔" and char_b in S) or (char_a in S and char_b =="⊔"):
        return 6.5

#Literals:
    if (char_a in UL and char_b in LL) or (char_a in LL and char_b in UL):
        return 3
    elif (char_a in UL and char_b in DD) or (char_a in DD and char_b in UL):
        return 8
    elif (char_a in UL and char_b in S) or (char_a in S and char_b in UL):
        return 8
    elif (char_a in UL and char_b in WS) or (char_a in WS and char_b in UL):
        return 8

    if (char_a in LL and char_b in DD) or (char_a in DD and char_b in LL):
        return 8
    elif (char_a in LL and char_b in S) or (char_a in S and char_b in LL):
        return 8
    elif (char_a in LL and char_b in WS) or (char_a in WS and char_b in LL):
        return 8

    if (char_a in DD and char_b in S) or (char_a in S and char_b in DD):
        return 7
    elif (char_a in DD and char_b in WS) or (char_a in WS and char_b in DD):
        return 7

    if (char_a in S and char_b in WS) or (char_a in WS and char_b in S):
        return 7

    if (char_a in UL and char_b in UL) or (char_a in LL and char_b in LL) or (char_a in DD and char_b in DD) or (char_a in WS and char_b in WS) or (char_a in S and char_b in S):
        return 0.5
    return 1.0

AZWeightedLevenshtein = AZWeightedLevenshtein(
    substitution_cost_fn=substitution_cost,
    insertion_cost_fn=insertion_cost,
    deletion_cost_fn=deletion_cost)

In [ ]:
def main(limlength,maxlen,genlevelcostmultiplier, minpatmultiplier,acc_threshold,lencomparison,pvalue,smallnfpvalue, smallnfnvalue):
  df = pd.read_csv(r'dirty_beers.csv',sep=',')
  df = df.astype(str)
  testlist=pd.Series.to_dict((df['id'].value_counts()))
  testlist

  b.clear()
  g.clear()
  coverage.clear()

  for word in testlist.items():
    wordlist=abstractwords(word[0])
    patterns=getpatterns(wordlist,word[0],limlength,maxlen)
    getdict=getdictionary(patterns,word[1])
  print("# of total patterns:",len(b))
  final=getscore(b,genlevelcostmultiplier,minpatmultiplier,acc_threshold,lencomparison,pvalue,smallnfpvalue, smallnfnvalue)

In [ ]:
default=(input)("would you like to use default values? (Y/N)")
if default=='Y':
   limlength=15
   maxlen=40
   genlevelcostmultiplier=1.2
   minpatmultiplier=0.7
   acc_threshold=0.95
   lencomparison=20
   pvalue=0.05
   smallnfpvalue=1.5
   smallnfnvalue=8
elif default =='N':
  limlength=int(input('limlength= (default value is 15)'))
  maxlen=int(input('maxlen= (default value is 40)'))
  genlevelcostmultiplier=float(input('genlevelcostmultiplier= (default value is 1.2)'))
  minpatmultiplier= float(input('minpatmultiplier= (default value is 0.7)'))
  acc_threshold= float(input('acc_threshold= (default value is 0.95)'))
  lencomparison=int(input('lencomparison= (default value is 20)')) #this is to limit the number of comparisons made for the similarity measure to improve performance. only relevant for large number of long strings (otherwise doesn't run)
  pvalue=float(input('pvalue= (default value is 0.05)'))
  smallnfpvalue=float(input('smallnfpvalue= (default value is 1.5)'))
  smallnfnvalue=float(input('smallnfnvalue= (default value is 8)'))
main(limlength,maxlen,genlevelcostmultiplier, minpatmultiplier,acc_threshold,lencomparison,pvalue,smallnfpvalue, smallnfnvalue)

would you like to use default values? (Y/N)N
limlength= (default value is 15)15
maxlen= (default value is 40)4
genlevelcostmultiplier= (default value is 1.2)1.2
minpatmultiplier= (default value is 0.7)0.7
acc_threshold= (default value is 0.95).95
lencomparison= (default value is 20)20
pvalue= (default value is 0.05).05
smallnfpvalue= (default value is 1.5)1.5
smallnfnvalue= (default value is 8)8
# of total patterns: 4652
0.6821576763485477 0.175 0.6821576763485477
0.27883817427385893 0.175 0.9609958506224067
0.03609958506224066 0.175 0.9609958506224067
0.002904564315352697 0.175 0.9609958506224067
{'ĐĐĐĐ': 1644, 'ĐĐĐ': 672, 'ĐĐ': 87, 'Đ': 7}
values: {'valid': ['1436', '1175', '1065', '1908', '1946', '1961', '1684', '1268', '1805', '1048', '1043', '1166', '1328', '1087', '1532', '1533', '1931', '1054', '2635', '1404', '1403', '1402', '2312', '1636', '1518', '1384', '1797', '1437', '1791', '1540', '1491', '1335', '1053', '1055', '1930', '1056', '2238', '2144', '2175', '2168', '1956', '17